In [ ]:
import torchaudio
torchaudio.set_audio_backend("soundfile")
from agents.tester import Tester
from utils.config import *


# YAML = 'configs/pre_trained/ResNet18.yaml'
# YAML = 'configs/pre_trained/ResNet34.yaml'
# YAML = 'configs/pre_trained/ResNetSE34V2.yaml'
# YAML = 'configs/pre_trained/ResNetSE34L.yaml'
YAML = 'configs/pre_trained/AutoSpeech.yaml'
# YAML = 'configs/pre_trained/VGGVox.yaml'
# YAML = 'configs/pre_trained/Cnn3D.yaml'
# YAML = 'configs/pre_trained/MobileNet.yaml'
# YAML = 'configs/pre_trained/SincNet-TIMIT.yaml'
# YAML = 'configs/pre_trained/SincNet-VoxCeleb2.yaml'
# YAML = 'configs/pre_trained/SincNet-AMI.yaml'

config = process_config(YAML)

agent = Tester(config)
agent.run()